In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.auto import tqdm

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

/home/tako/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#GPU 사용
device = torch.device("cuda:0")

In [13]:
path = 'data/'
train_data = pd.read_csv(path + 'train_data.csv',encoding='utf-8-sig')
test_data = pd.read_csv(path + 'test_data.csv',encoding='utf-8-sig')

In [5]:
train_dataset = []
test_dataset = []
for sen, label in zip(train_data['title'], train_data['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))
  train_dataset.append(data_train)
for sen, label in zip(test_data['title'],test_data['topic_idx']):
  data_test = []
  data_test.append(sen)
  data_test.append(str(label))
  test_dataset.append(data_test)

In [7]:

train = train_dataset
test = test_dataset
print(len(train), len(test))


27392 18262


In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.2
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  1e-4

In [10]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/tako/hykim/Dacon_classification/.cache/kobert_v1.zip
using cached model. /home/tako/hykim/Dacon_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

using cached model. /home/tako/hykim/Dacon_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle= False, num_workers=2)


In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)

#optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return acc



In [15]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    # evaluation

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
       with torch.no_grad():
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
    

    
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 1/428 [00:00<04:02,  1.76it/s]

epoch 1 batch id 1 loss 1.9450528621673584 train acc 0.15625


 47%|████▋     | 201/428 [00:49<00:55,  4.09it/s]

epoch 1 batch id 201 loss 0.33757150173187256 train acc 0.650497512437811


 94%|█████████▎| 401/428 [01:38<00:06,  4.16it/s]

...

 47%|████▋     | 201/428 [00:50<00:56,  3.98it/s]

epoch 4 batch id 201 loss 0.08756005018949509 train acc 0.9597325870646766


 94%|█████████▎| 401/428 [01:40<00:06,  4.00it/s]

epoch 4 batch id 401 loss 0.13086850941181183 train acc 0.9610738778054863


 18%|█▊        | 51/286 [00:04<00:19, 11.96it/s]